# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports



In [1]:
import numpy as np
import pandas as pd
import requests
import math
#from scipy import stats
from statistics import mean
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks



In [2]:
stocks = pd.read_csv(r"C:\Users\conta\OneDrive\Desktop\Project Pro\algorithmic-trading-python-master\algorithmic-trading-python-master\starter_files\sp_500_stocks.csv")
from secret import IEX_CLOUD_API_TOKEN

## Making Our First API Call



In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
#data.status_code
data



{'companyName': 'Apple Inc',
 'marketcap': 2564505772662,
 'week52high': 188.29,
 'week52low': 133.04,
 'week52highSplitAdjustOnly': 187.51,
 'week52lowSplitAdjustOnly': 130.87,
 'week52change': 0.04549787679525132,
 'sharesOutstanding': 16537754420,
 'float': 0,
 'avg10Volume': 92723784,
 'avg30Volume': 77472525,
 'day200MovingAvg': 156.55,
 'day50MovingAvg': 171.19,
 'employees': 149634,
 'ttmEPS': 6.09,
 'ttmDividendRate': 0.9422891121717385,
 'dividendYield': 0.00611004030398842,
 'nextDividendDate': '',
 'exDividendDate': '2022-07-27',
 'nextEarningsDate': '2022-10-25',
 'peRatio': 25.644648803455418,
 'beta': 1.287848288341468,
 'maxChangePercent': 60.619610711126604,
 'year5ChangePercent': 3.0906474183569257,
 'year2ChangePercent': 0.409602527989601,
 'year1ChangePercent': 0.0345759424562428,
 'ytdChangePercent': -0.13586182933565077,
 'month6ChangePercent': -0.00305041607985062,
 'month3ChangePercent': 0.168714001079474,
 'month1ChangePercent': -0.10940864679080564,
 'day30Chan

## Parsing Our API Call



In [4]:
data['year1ChangePercent']

0.0345759424562428

## Executing A Batch API Call & Building Our DataFrame



In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
#for symbol_string in symbol_strings:
    #print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'DISCA' or symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW': # there are some errors with these stocks
            continue
        df = df.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,137,-0.220582,N/A
1,AAL,14.54,-0.301337,N/A
2,AAP,171.53,-0.112069,N/A
3,AAPL,162.85,0.0347111,N/A
4,ABBV,143.96,0.341927,N/A
...,...,...,...,...
496,YUM,114.97,-0.106104,N/A
497,ZBH,119.56,-0.171473,N/A
498,ZBRA,307.5,-0.497591,N/A
499,ZION,60.78,0.0074366,N/A


## Removing Low-Momentum Stocks




In [7]:
df.sort_values('One-Year Price Return', ascending = False, inplace = True)
df = df[:50];
df.reset_index(inplace = True)
df

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,81.41,2.35825,N/A
1,147,DVN,74.36,1.57048,N/A
2,355,OXY,69.1,1.48531,N/A
3,315,MRO,28.64,1.23651,N/A
4,89,CF,107.08,1.23227,N/A
5,109,COP,122.63,1.03495,N/A
6,42,APA,42.48,0.996737,N/A
7,159,EOG,131.88,0.925274,N/A
8,225,HRB,45.92,0.912226,N/A
9,492,XOM,98.62,0.82901,N/A


## Calculating the Number of Shares to Buy



In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio:")
    
    try:
        float(portfolio_size)
        
    except ValueError:
            print("That's not a number! \nPlease try again")
            portfolio_size = input("Enter the size of your portfolio:")
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:1000000
1000000


In [9]:
position_size = float(portfolio_size) / len(df.index)
for i in range(0, len(df['Ticker'])):
    df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / df['Price'][i])
df

C:\Users\conta\anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,81.41,2.35825,245
1,147,DVN,74.36,1.57048,268
2,355,OXY,69.1,1.48531,289
3,315,MRO,28.64,1.23651,698
4,89,CF,107.08,1.23227,186
5,109,COP,122.63,1.03495,163
6,42,APA,42.48,0.996737,470
7,159,EOG,131.88,0.925274,151
8,225,HRB,45.92,0.912226,435
9,492,XOM,98.62,0.82901,202


## Building a Better (and More Realistic) Momentum Strategy



In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'DISCA' or symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW': # there are some errors with these stocks
            continue
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,138.25,N/A,-0.224305,N/A,0.0282253,N/A,0.154434,N/A,-7.6553e-05,N/A,N/A
1,AAL,14.21,N/A,-0.302471,N/A,-0.0246253,N/A,0.0279581,N/A,-0.106202,N/A,N/A
2,AAP,176.06,N/A,-0.112115,N/A,-0.142121,N/A,0.0125945,N/A,-0.149837,N/A,N/A
3,AAPL,162.34,N/A,0.0348175,N/A,-0.0030373,N/A,0.169765,N/A,-0.109879,N/A,N/A
4,ABBV,141.65,N/A,0.351374,N/A,-0.0555123,N/A,0.00296999,N/A,-0.0291228,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,115.36,N/A,-0.104034,N/A,-0.013524,N/A,0.0285574,N/A,-0.0357102,N/A,N/A
497,ZBH,120.84,N/A,-0.176205,N/A,-0.0166964,N/A,0.0623479,N/A,-0.00715226,N/A,N/A
498,ZBRA,298.97,N/A,-0.50327,N/A,-0.243116,N/A,0.00147173,N/A,-0.15541,N/A,N/A
499,ZION,60.16,N/A,0.00765166,N/A,-0.0959584,N/A,0.133293,N/A,-0.00907743,N/A,N/A


## Calculating Momentum Percentiles



In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe[change_col] = hqm_dataframe[change_col].astype(float) #converts change_col data type to float
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

# Print each percentile score to make sure it was calculated properly
#for time_period in time_periods:
#    print(hqm_dataframe[percentile_col])

#Print the entire DataFrame    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,138.25,N/A,-0.224305,0.255489,0.028225,0.674651,0.154434,0.832335,-0.000077,0.868263,N/A
1,AAL,14.21,N/A,-0.302471,0.137725,-0.024625,0.53493,0.027958,0.403194,-0.106202,0.221557,N/A
2,AAP,176.06,N/A,-0.112115,0.45509,-0.142121,0.249501,0.012595,0.353293,-0.149837,0.0778443,N/A
3,AAPL,162.34,N/A,0.034818,0.704591,-0.003037,0.586826,0.169765,0.874251,-0.109879,0.211577,N/A
4,ABBV,141.65,N/A,0.351374,0.91018,-0.055512,0.479042,0.002970,0.315369,-0.029123,0.732535,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,115.36,N/A,-0.104034,0.46507,-0.013524,0.56487,0.028557,0.413174,-0.035710,0.702595,N/A
497,ZBH,120.84,N/A,-0.176205,0.339321,-0.016696,0.558882,0.062348,0.540918,-0.007152,0.850299,N/A
498,ZBRA,298.97,N/A,-0.503270,0.0299401,-0.243116,0.0818363,0.001472,0.303393,-0.155410,0.0718563,N/A
499,ZION,60.16,N/A,0.007652,0.658683,-0.095958,0.357285,0.133293,0.792415,-0.009077,0.840319,N/A


## Calculating the HQM Score



In [12]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,138.25,N/A,-0.224305,0.255489,0.028225,0.674651,0.154434,0.832335,-0.000077,0.868263,0.657685
1,AAL,14.21,N/A,-0.302471,0.137725,-0.024625,0.53493,0.027958,0.403194,-0.106202,0.221557,0.324351
2,AAP,176.06,N/A,-0.112115,0.45509,-0.142121,0.249501,0.012595,0.353293,-0.149837,0.0778443,0.283932
3,AAPL,162.34,N/A,0.034818,0.704591,-0.003037,0.586826,0.169765,0.874251,-0.109879,0.211577,0.594311
4,ABBV,141.65,N/A,0.351374,0.91018,-0.055512,0.479042,0.002970,0.315369,-0.029123,0.732535,0.609281
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,115.36,N/A,-0.104034,0.46507,-0.013524,0.56487,0.028557,0.413174,-0.035710,0.702595,0.536427
497,ZBH,120.84,N/A,-0.176205,0.339321,-0.016696,0.558882,0.062348,0.540918,-0.007152,0.850299,0.572355
498,ZBRA,298.97,N/A,-0.503270,0.0299401,-0.243116,0.0818363,0.001472,0.303393,-0.155410,0.0718563,0.121756
499,ZION,60.16,N/A,0.007652,0.658683,-0.095958,0.357285,0.133293,0.792415,-0.009077,0.840319,0.662176


## Selecting the 50 Best Momentum Stocks



In [13]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,81.64,N/A,2.326013,0.992016,0.854007,0.99002,0.226168,0.922156,0.081274,0.972056,0.969062
1,ALB,317.91,N/A,0.316578,0.896208,0.637790,0.988024,0.354028,0.984032,0.060146,0.964072,0.958084
2,HRB,46.99,N/A,0.885395,0.976048,0.909828,0.992016,0.350496,0.98004,-0.004971,0.858283,0.951597
3,COG,23.28,N/A,0.351835,0.912176,0.184890,0.934132,0.257510,0.946108,0.254676,0.992016,0.946108
4,UNM,40.78,N/A,0.581582,0.952096,0.499572,0.984032,0.192036,0.898204,0.020425,0.934132,0.942116
5,NLSN,28.92,N/A,0.415290,0.932136,0.228297,0.96008,0.242692,0.942116,0.012059,0.918164,0.938124
6,AES,28.27,N/A,0.161800,0.818363,0.227257,0.958084,0.409163,0.99002,0.081542,0.974052,0.93513
7,GPC,156.71,N/A,0.370672,0.916168,0.326782,0.982036,0.207485,0.91018,0.004833,0.902196,0.927645
8,CI,284.55,N/A,0.397253,0.928144,0.271863,0.97006,0.184329,0.89022,-0.014249,0.820359,0.902196
9,SRE,178.40,N/A,0.325283,0.902196,0.127749,0.862275,0.177673,0.884232,0.027988,0.944112,0.898204


## Calculating the Number of Shares to Buy



In [14]:
portfolio_input()

Enter the size of your portfolio:1000000


In [15]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
hqm_dataframe
    

C:\Users\conta\anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,81.64,244,2.326013,0.992016,0.854007,0.99002,0.226168,0.922156,0.081274,0.972056,0.969062
1,ALB,317.91,62,0.316578,0.896208,0.637790,0.988024,0.354028,0.984032,0.060146,0.964072,0.958084
2,HRB,46.99,425,0.885395,0.976048,0.909828,0.992016,0.350496,0.98004,-0.004971,0.858283,0.951597
3,COG,23.28,859,0.351835,0.912176,0.184890,0.934132,0.257510,0.946108,0.254676,0.992016,0.946108
4,UNM,40.78,490,0.581582,0.952096,0.499572,0.984032,0.192036,0.898204,0.020425,0.934132,0.942116
5,NLSN,28.92,691,0.415290,0.932136,0.228297,0.96008,0.242692,0.942116,0.012059,0.918164,0.938124
6,AES,28.27,707,0.161800,0.818363,0.227257,0.958084,0.409163,0.99002,0.081542,0.974052,0.93513
7,GPC,156.71,127,0.370672,0.916168,0.326782,0.982036,0.207485,0.91018,0.004833,0.902196,0.927645
8,CI,284.55,70,0.397253,0.928144,0.271863,0.97006,0.184329,0.89022,-0.014249,0.820359,0.902196
9,SRE,178.40,112,0.325283,0.902196,0.127749,0.862275,0.177673,0.884232,0.027988,0.944112,0.898204


## Formatting Our Excel Output



In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File


In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', percent_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()